# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# APK Files to load
file_paths = (
#     "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.0.63-2076_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.5.72-2077_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.89.0.66-2078_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.89.7.78-2085_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
#     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
    "../../../Downloads/com.instagram.android_156.0.0.26.109-240726452_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726384_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
    "../../../Downloads/com.instagram.android_157.0.0.37.120-242168935_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168922_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_156.0.0.26.109-240726452_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_157.0.0.37.120-242168935_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `named_c_decs` in form of a tuple of Strings
* `named_m_decs` in form of a tuple of MethodDeclarations
* `named_f_decs` in form of a tuple of FieldDeclarations

In [11]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.4zu',
    'COMMENT_LONGCLICK_HANDLER': 'X.2wP',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.3Ud',
    'DM_HOLDER': 'X.A5p',
    'DM_REPLY': 'X.3PL',
    'DM_VOICEMESSAGE': 'X.AK1',
    'FEEDITEM': 'X.1T9',
    'FRAGMENT': 'X.1NZ',
    'HEADERGENERATOR': 'X.0Qj',
    'HTTPREQUESTTASKFACTORY': 'X.0p6',
    'INAPP_NOTIFICATION': 'X.6ip',
    'IGTV_VIEWER_FRAGMENT': 'X.45I',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.11y',
    'JACKSON_CREATOR': 'X.0iH',
    'LIVE_POST': 'X.271',
    'MAINFEED': 'X.1O5',
    'MEDIAITEM': 'X.1TF',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1lf',
    'NOTIFICATION_HANDLER': 'X.2Eb',
    'PHOTOTIMERCONTROLLER': 'X.3KY',
    'POSTDETAIL': 'X.1bB',
    'POSTGESTURE_SIMPLEPOST': 'X.1uq',
    'POSTGESTURE_CAROUSELIMAGE': 'X.22U',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2Qj',
    'POSTVIEW': 'X.1bG',
    'PROFILE': 'X.0kh',
    'PROFILEDETAIL': 'X.3JG',
    'PROFILEPIC_LONGCLICK': 'X.3JN',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.29P',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0mq',
    'NOTIFICATION_RUNNABLE': 'X.2n9',
    'SERIALSCHEDULER': 'X.0i8',
    'SESSION': 'X.0LC',
    'SIMPLE_EXECUTOR': 'X.4Fd',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1VW',
    'VIDEOSETTINGSMANAGER': 'X.1eq',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [12]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.4zu', '<init>', 'X.1y9', 'X.6ee', 'X.0LC', 'android.view.View', 'X.4zh', 'X.503'),
    'COMMENT_ONLONGCLICK': MethodDec('X.2wP', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1FD', 'androidx.fragment.app.FragmentActivity', 'X.0LC', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0LC'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0LC'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3Ud', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.A5p', 'A01'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.3PL', 'A0J', 'X.3PL', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.3PL', 'A0F', 'X.3PL'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1T9', 'A00', 'X.0iZ'),
    'FRAGMENT_GETACTIVITY': MethodDec('X.1NZ', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0Qj', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0p6', 'A03', 'java.lang.String', 'X.0LC'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.6ip', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.45I', 'A00', 'X.45I'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.45I', 'A0B', 'X.45I'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.11y', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0iH', 'A04', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0iH', '<init>', 'X.Dzq'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.271', 'A00', 'X.0jO', 'X.272'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'AKr'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1O5', 'configureActionBar', 'X.1Ik'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1TF', 'A01', 'X.0iZ', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0jO', 'X.1TF'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1lf', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2Eb', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.3KY', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1bB', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.1uq', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.22U', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2Qj', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1bG', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.3JG', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0kh', 'A04', 'X.0LC', 'X.0j9'),
    'PROFILE_ONLONGCLICK': MethodDec('X.3JN', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.29P', 'A0T', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1K', 'X.1rQ'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0p', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0T', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A10', 'X.1nF', 'X.1rQ'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0mq', 'A00', 'X.0lo', 'X.0yQ'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.2n9', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0i8', 'A02', 'X.0pY'),
    'SESSION_GETTOKEN': MethodDec('X.0LC', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.4Fd', 'onFail', 'X.2ED'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.4Fd', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.4Fd', '<init>', 'X.0LC'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1VW', 'A03', 'X.0jO', 'X.0j8'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1eq', 'A00', 'X.0LC'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [13]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1Ij', 'A08'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.1y9', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.2wP', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1X0', 'A0X'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3Ud', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.A5f', 'A07'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.AK1', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1T9', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0pX', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.45I', 'A0E'),
    'MAINFEED_SESSION': FieldDec('X.1O5', 'A0S'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1Sl', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1rQ', 'A09'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1rQ', 'A08'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.10X', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1dI', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.6ip', 'A00'), FieldDec('X.2n9', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.6ip', 'A01'), FieldDec('X.2n9', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1qp', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1qs', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.3JJ', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.3JJ', 'A00'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.3JJ', 'A01'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.3JJ', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.3JL', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.10V', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.2lz', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1K'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1dE', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1rQ', 'A0D')
}

In [14]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [15]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [16]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [17]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [18]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	X.0Qj#A00(android.content.Context) -> X.0Q4#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	X.1lf#A01() -> X.1jR#A01()
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A10(X.1nF, X.1rQ) -> com.instagram.reels.fragment.ReelViewerFragment#A0y(X.1lh, X.1rX)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback)
+ Found single candidate for Method. Considering it a match 
	X.3PL#A0F(X.3PL) -> X.3AA#A0I(X.3AA)
+ Found single candidate for Method. Considering it a match 
	com

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [19]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

~ Unique String not used in single class anymore. Change! (X-Tigon-Is-Retry)
~ Unique String not used in single class anymore. Change! (error_on_body)
~ Unique String not used in single class anymore. Change! (error_on_eom)
~ Unique String not used in single class anymore. Change! (Error is null)
~ Unique String not used in single class anymore. Change! (error_on_error)
~ Unique String not used in single class anymore. Change! (error_on_response)
~ Unique String not used in single class anymore. Change! (cover_frame_url)
~ Unique String not used in single class anymore. Change! (broadcast_owner)
.. Considering ambiguous match by UniqueString as match with a majority of 0.67 (LX/0mq; -> LX/0l5;)
.. Considering ambiguous match by UniqueString as match with a majority of 0.83 (LX/10V; -> LX/1eA;)
+ Found single candidate for Class. Considering it a match! 
	LX/0LC; -> LX/0LH;
+ Found single candidate for Class. Considering it a match! 
	LX/0i8; -> LX/0hv;
+ Found single candidate for Clas

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [20]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/4zu; -> LX/5EH;
+ Found single candidate for Class. Considering it a match! 
	LX/2wP; -> LX/3ZZ;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3Ud; -> LX/3OZ;
+ Found single candidate for Class. Considering it a match! 
	LX/0iH; -> LX/0i4;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/1uq; -> LX/1sH;
* Found multiple candidates for Class LX/22U;
* Found multiple candidates for Class LX/2Qj;
+ Found single candidate for Class. Considering it a match! 
	LX/1bG; -> LX/1WS;
+ Found single candidate for Class. Considering it a match! 
	LX/0kh; -> LX/0k

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [21]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.A5p#A01()
Class not resolved for Method X.3KY#A01()
Class not resolved for Method X.22U#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2Qj#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	X.2wP#onLongPress(android.view.MotionEvent) -> X.3ZZ#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1FD, androidx.fragment.app.FragmentActivity, X.0LC, android.os.Bundle) -> com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1F7, androidx.fragment.app.FragmentActivity, X.0LH, android.

Could resolve 0 new Classes, 12 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [22]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0j8; -> LX/0iv;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1ZI; -> LX/1P2;
+ Found single candidate for Class. Considering it a match! 
	LX/272; -> LX/26U;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0pZ; -> LX/0tt;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a match!

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [23]:
print(len(accumulator.matching_cs), "/", len(named_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

69 / 45 Classes were matched
52 / 58 Methods were matched
32 / 37 Fields were matched

Matching Classes:
• X.0Qj -> X.0Q4
• X.1lf -> X.1jR
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.3PL -> X.3AA
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1O5 -> X.1Ip
• X.1eq -> X.1gG
• X.29P -> X.2CI
• X.2n9 -> X.2c8
• X.3JG -> X.3C8
• X.4Fd -> X.4LK
• X.6ip -> X.6lv
• X.0LC -> X.0LH
• X.0i8 -> X.0hv
• X.0mq -> X.0l5
• X.0p6 -> X.0q5
• X.0pX -> X.0tm
• X.10V -> X.1eA
• X.10X -> X.1eB
• X.11y -> X.10T
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Ij -> X.1Hv
• X.1NZ -> X.1IC
• X.1T9 

• X.1TF#A01(X.0iZ, boolean) -> X.1NK#A01(X.0iM, boolean)
• com.instagram.feed.media.Media__JsonHelper#A00(X.0jO, X.1TF) -> com.instagram.feed.media.Media__JsonHelper#A00(X.0jB, X.1NK)
• X.2Eb#A01(android.content.Context, java.lang.CharSequence, int) -> X.2KH#A01(android.content.Context, java.lang.CharSequence, int)
• X.1bG#A01(X.1J5, X.1JE, X.1TF, X.1rf, int, X.1qs, java.lang.String) -> X.1WS#A01(X.1QE, X.1QM, X.1NK, X.1qp, int, X.1q6, java.lang.String)
• com.instagram.reels.fragment.ReelViewerFragment#A0T(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0R(com.instagram.reels.fragment.ReelViewerFragment)
• X.0mq#A00(X.0lo, X.0yQ) -> X.0l5#A02(X.1cr, java.lang.String, com.facebook.proxygen.HTTPRequestHandler, com.facebook.proxygen.JniHandler, com.facebook.proxygen.NativeReadBuffer, com.facebook.proxygen.TraceEventContext)
• X.0i8#A02(X.0pY) -> X.0hv#A02(X.0tn)
• X.0LC#getToken() -> X.0LH#getToken()
• X.1VW#A03(X.0jO, X.0j8) -> X.1Nx#A

In [24]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(named_c_decs.values(), r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.5EH', '<init>', 'X.22x', 'X.9Hx', 'X.0LH', 'android.view.View', 'X.5Dw', 'X.5EQ'),
    MethodDec('X.3ZZ', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1F7', 'androidx.fragment.app.FragmentActivity', 'X.0LH', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LC),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LC),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerCont

In [25]:
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


c_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [26]:
%autoreload

c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LC)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LC)
No matching Method found for X.A5p#A01()
No matching Method found for X.3KY#A01()
No matching Method found for X.22U#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2Qj#onDoubleTap(android.view.MotionEvent)
No matching Field found for com.instagram.direct.visual.DirectVisualMessageViewerController#A0D
No matching Field found for com.instagram.direct.visual.DirectVisualMessageViewerController#mRootView
No matching Field found for X.45I#A0E
No matching Field found for X.1qp#A06
No matching Field found for X.1qs#A02


In [27]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.5EH")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3ZZ")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.3OZ")

    override val DM_HOLDER = /* TODO */ ClassDec("X.A5p")

    override val DM_REPLY = ClassDec("X.3AA")

    override val DM_VOICEMESSAGE = ClassDec("X.AYP")

    override val FEEDITEM = ClassDec("X.1NJ")

    override val FRAGMENT = ClassDec("X.1IC")

    override val HEADERGENERATOR = ClassDec("X.0Q4")

 

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1Hv"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A08")

    @FieldClass(["X.22x"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3ZZ"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1OU"])
    override val COMMENT_TEXT = VariableDec<String>("A0X")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = /* TODO */ VariableDec<Any>("A0D")

    @

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```